# Лабораторная работа №1. Кластерный анализ

### 1. Обработка данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering, KMeans
from scipy.cluster import hierarchy

%matplotlib inline 
# авто вывод графиков (отпадает необходимость писать plt.show())

Указываем названия признаков

In [2]:
features = ['X20', 'X22', 'X29', 'X37', 'X38'] #список переменных
f_len = len(features) #в переменную f_len записываем кол-во признаков

Считываем данные

In [3]:
#считываем данные из data_lab1.xlsx (файл, который содержит все строки и выбранные признаки)
data = pd.read_excel('11.xlsx',0)
data.head() # метод head() выводит первые 5 записей 

FileNotFoundError: [Errno 2] No such file or directory: '11.xlsx'

In [ ]:
data.shape #метод shape возвращает размерность матрицы объект-свойство

Стандартизируем данные

In [ ]:
index = data.index #записываем в переменную index названия строк исходной матрицы
columns = data.columns #записываем в переменную columns названия столбцов исходной матрицы

#стандартизируем данные: вычитаем среднее и делим на стандартное отклонение
scaled = (data - data.mean(axis=0))/data.std() 

#создаем датафрейм из scaled со столбцами columns и индексами index
scaled_data = pd.DataFrame(scaled, columns=columns, index=index) 
scaled_data.head()

### 2. Построение дендрограмм

In [ ]:
methods = ['complete', 'ward']
q = [3,3]
i = 0
#для каждого из методов в methods проводим кластеризацию и строим дендрограмму
for method in methods:
    #проводим кластеризацию
    Z = hierarchy.linkage(scaled_data, method=method)
    
    #строим дендрограмму 
    plt.figure(figsize=(10,6))
    hierarchy.dendrogram(Z, color_threshold=q[i], labels=scaled_data.index, leaf_font_size=10)
    plt.title('{} method'.format(method))
    i = i+1

In [ ]:
labels = ['Оренбургский','г.Оренбург','г.Орск']
#удаляем эти строки из исходного датафрейма
data.drop(labels=labels, inplace=True)
index = data.index

#заново стандартизируем
scaled = (data - data.mean(axis=0))/data.std()
scaled_data = pd.DataFrame(scaled, columns=columns, index=index) 
scaled_data.head()



Если видим, что такое разбиение нас не устраивает (выброс образует целый кластер, например), то необходимо удалить выбросы и провести кластеризацию заново.

In [ ]:
#заново смотрим дендрограммы, принимаем решение, на сколько кластеров делить
methods = ['complete', 'ward']
q = [4,7]
i = 0
#для каждого из методов в methods проводим кластеризацию и строим дендрограмму
for method in methods:
    #проводим кластеризацию
    Z = hierarchy.linkage(scaled_data, method=method)
    
    #строим дендрограмму 
    plt.figure(figsize=(10,6))
    hierarchy.dendrogram(Z, color_threshold=q[i], labels=scaled_data.index, leaf_font_size=10)
    plt.title('{} method'.format(method))
    i = i+1

In [ ]:
#если такое разбиение нас устраивает, продолжаем кластеризацию

### 3. Проведение кластеризации тремя методами

Т.к. метод hierarchy.linkage не возвращает метки(номера) кластеров, то необходимо провести кластеризацию с помощью AgglomerativeClustering
<br>
<br>
**n_clusters** - кол-во кластеров, которое мы выбираем самомтоятельно
<br>
**linkage** - выбранные метод

In [ ]:
CLUSTER_METHODS = ["complete", "ward", "kmeans"]

#для каждого из методов выбрать кол-во кластеров 
N_CLUSTERS = {
    "complete" : 3,
    "ward"     : 3,
    "kmeans"  : 3,
}

In [ ]:
#проводим кластеризацию методом полных связей

# создаем объект класса AgglomerativeClustering, указываем для этого объекта значения параметров
# т.е. n_clusters - кол-во кластеров, linkage - метод кластеризации
# называем созданный объект complete

complete = AgglomerativeClustering(n_clusters=N_CLUSTERS['complete'], linkage='complete')

#методы, которые можно указывать в linkage: “ward”, “complete”, “average”

# с помощью метода .fit "настраиваем" объект под наши данные, 
# т.е. проводим кластеризацию наших данных с параметрами, которые мы указали при создании complete 
complete.fit(scaled_data);

In [ ]:
#проводим кластеризацию методом Уорда
ward = AgglomerativeClustering(n_clusters=N_CLUSTERS['ward'], linkage='ward')
ward.fit(scaled_data);

Для проведения кластеризации методом K-means необходимо воспользоваться KMeans из sklearn.cluster
<br>
<br>
**random_state** - параметр, принимающий любое числовое значение. Необходим для воспроизведения результатов

In [ ]:
#проводим кластеризацию методом K-means
kmeans = KMeans(n_clusters=N_CLUSTERS['kmeans'], random_state=17)
kmeans.fit(scaled_data);

In [ ]:
scaled_data['complete'] = complete.labels_
scaled_data['ward'] = ward.labels_
scaled_data['kmeans'] = kmeans.labels_
scaled_data.head()

Для каждого из созданных шагом ранее объектов с помощью атрибута **.labels_** можно получить метки кластеров для каждой из проведенных кластеризаций.
<br>
**complete.labels_** - для метода полных связей
<br>
**ward.labels_** - для метода Уорда
<br>
**kmeans.labels_** - для K-Means

### 4. Работа с метками кластеров. Построение графиков средних значений для каждого кластера по каждому признаку

In [ ]:
#ниже объявлена функция, которая возвращает список из средних по каждому столбцу.
#т.е. она формирует массив из средних для метода method c кол-вом кластеров n_clust

def mean_df(method, n_clust):
    mean_data = np.array([]).reshape(0, f_len+1)
    for n in range(n_clust): #номер кластера
        tmp = [] #список для средних по каждому признаку
        for j in range(f_len):
            tmp.append(scaled_data[scaled_data[method] == n].iloc[:, j].mean())
        tmp.append(scaled_data[scaled_data[method] == n].shape[0])
        mean_data = np.vstack((mean_data, np.array(tmp).reshape(1, f_len+1)))
    return mean_data

Формируем датафрейм из средних значений для каждого кластера по каждому признаку с использованием функции mean_df, описанный выше

In [ ]:
columns = features + ['count']

# Создаем словарь из списком по имени means, в котором имена ключей будут Имя_Метода,
# а каждому ключу будет соответствовать датафрейм из средних значений по каждому кластеру

means = {}
dfs = []

path = "means.xlsx" # создаем переменную path, которой присваиваем название файла для записи средних
writer = pd.ExcelWriter(path) # вызываем метод ExcelWriter библиотеки pandas 


for method, n in N_CLUSTERS.items():
    #Заполняем словарь
    means[method] = pd.DataFrame(
        mean_df(method, n), #ищем среднее значение признаков с помощью функции mean_df
        columns=columns, 
        index=["{}_{}".format(method, i) for i in range(n)]
    )

    #запись средних значений в в файл means на разные листв
    #файл создается в той же директории, где лежит ноутбук с кодом
    means[method].to_excel(writer, sheet_name = '{} means'.format(method))

In [ ]:
#Получился словарь means с ключами 'complete', 'ward', 'kmeans'
#Каждому ключу соответствует датафрейм из 6 столбцов и стольких строк, 
#сколько кластеров в данном конкретном методе

mean_data = pd.DataFrame()
for method in CLUSTER_METHODS:
    mean_data = mean_data.append(means[method])
mean_data

In [ ]:
#строим графики средних значений признаков по каждому кластеру

#цикл выполняется для каждого элемента из списка CLUSTER_METHODS т.е. для 'complete', 'ward', 'kmeans'
for method in CLUSTER_METHODS:
    # в cur_mean записываются средние для текущего метода (по которому проходим на данной итерации цикла)
    cur_mean = means[method] 
    plt.figure(figsize=(6,4)) #формируем область для построения графиков под каждый из методов
    for n in range(cur_mean.shape[0]): #для каждого кластера из данного метода
        
        #plt.plot - для каждого кластера из данного метода строится обычный линейный график, где 
        #x=features (т.е. названия признаков) 
        #y=cur_mean.iloc[n, :-1] - т.е. n-ая строка средних 
        #(n - номер кластера, для которого на данной итерации цикла строим график) 
        #берем строку без последнего столбца (там count лежит, он нам не нужен), т.е. пишем ":-1" в методе iloc
        
        plt.plot(features, cur_mean.iloc[n, :-1].values, marker='o', label='cluster {}'.format(n))
    plt.legend(loc = 'upper left') #делаем легенду графиа
    plt.title('{} method'.format(cur_mean.index[0][:-2])) #название графика

Создадим словарь cluster_dict из списков, в котором ключами будут **Название Метода__Номер Кластера**
<br>
<br>
Например, **complete_0** - элемент словаря, по которому можно получить строки, соответствующие кластеру 0 для метода полных связей

In [ ]:
cluster_dict = {}


path = "members.xlsx" # создаем переменную path, которой присваиваем название файла для записи объектов кластеров
writer = pd.ExcelWriter(path) # вызываем метод ExcelWriter библиотеки pandas 

for method, n in N_CLUSTERS.items():
    print('{} method\n'.format(method)) # выводим заголовки методов по шаблону 
    dfs = []  
    for i in range(n): # для каждого кластера проходимся по объектам и записываем их в словарь 
        cluster_dict['{}_{}'.format(method, i)] = \
           scaled_data[scaled_data[method]==i][features]
        
        #запись членов кластеров в файл members на лист с именем Метод_НомерКластера
        #файл создается в той же директории, где лежит ноутбук с кодом
        cluster_dict['{}_{}'.format(method, i)]\
        .to_excel(writer, sheet_name = '{}_{}'.format(method, i))
        
        print ('Состав кластера {}: {}'
              .format(i, cluster_dict['{}_{}'.format(method, i)].index.values))      
    print ('\n')

### 5. Функционал ошибки

In [ ]:
def Q(method_num, mean_data_row):
    q = 0 # Обнуляем функционал качества разбиения 
    for i in range(len(method_num)): # проходимся по каждому элементу кластера
        
        # Во временную переменную записываем промежуточные значения суммы, чтобы потом добавлять их к Q
        tmp = 0
        for j, feature in zip(range(len(features)), features):
            
            #проходя по каждому признаку, считаем сумму квадратов расстояний от значения объекта до центра кластера
            tmp = tmp + (method_num.iloc[i, j] - mean_data_row[feature][0])**2 
            
            # Прибавляем к Q
        q = q + tmp
    #возвращаем конечную сумму
    return q

In [ ]:
#создаем пустой словарь, в который будем записывать значения функционалов ошибки для каждого метода
Q_dict = {}

#k - номер строки в датафрейме mean_data (выведен выше)
k = 0

for method, n in N_CLUSTERS.items():
    #временная переменная, нужна для суммирования функционала ошибки для каждого кластера в рамках одного метода
    tmp = 0
    #для каждого кластера (их всего n) в данном методе method выполняются действия в теле цикла
    for i in range(n):
        #в переменную tmp прибавляется значение Q для текущего номера кластера i (изменяется от 0 до n-1)
        #в Q передаем в качестве первого аргумента - датафрейм, содержащий члены текущего кластера
        #в качестве второго аргумента - строку из mean_data с индексом k (там записаны средние значения для данного кластера i)
        tmp += Q(cluster_dict['{}_{}'.format(method, i)], pd.DataFrame(mean_data.iloc[k]).T)
        #увеличиваем k на 1 (чтобы перейти к следующей строке mean_data)
        k += 1   
    #после того, как прошли по всем кластерам в рамках метода method и просуммировали функционалы ошибки в переменной tmp, запишем результирующую сумму в словарь
    Q_dict['Q_{}'.format(method)] = round(tmp, 3)

In [ ]:
Q_dict